# <center> Querying Wikidata with SPARQL</center>
<font color="#5F6A6A"> <center> Cristina Sarasua (University of Zurich) </center> </font>
   
   
  <hr>

## Introduction

Wikidata's data can be accessed in many different ways. Depending on what exactly you would like to access (e.g., item descriptions, or page edit history) you may:
* query the knowledge base live using the RDF-based query language SPARQL 
* download a data dump (i.e., containing a snapshot of the data at a certain point in time)
* query the knowledge base and its edit history live via the Wikidata's instance of the Media Wiki API 
* retrieve concrete item descriptions as Linked Data executing an HTTP GET request

In this hands-on we will focus on querying data items of the knowledge base using the <a href="https://www.w3.org/TR/sparql11-query/">SPARQL query language</a> --- the language that is used to query <a href="https://www.w3.org/DesignIssues/LinkedData.html">Linked Data</a> and <a href="https://cacm.acm.org/magazines/2016/9/206254-a-new-look-at-the-semantic-web/fulltext">Semantic Web data</a> in general.          

## Wikidata Documentation

If you would like to look up further information about Wikidata, I would recommend you read the article at the Communications of the ACM journal, by Denny Vrandečić and Markus Krötzsch: 
<a href="https://cacm.acm.org/magazines/2014/10/178785-wikidata/fulltext">Wikidata: a Free Collaborative, Knowledge Base</a>, and the following information: 

Wikidata's data (model and data access) 

* **Wikidata's Help Page** <br> 
https://www.wikidata.org/wiki/Help:Contents

* **How to access data in Wikidata (general page)** <br>
https://www.wikidata.org/wiki/Wikidata:Data_access

Querying Wikidata with SPARQL

* **Wikidata's SPARQL Query Service** <br>
https://query.wikidata.org/

* **Instructions about how to use the Query Service** <br> https://www.wikidata.org/wiki/Wikidata:SPARQL_query_service/A_gentle_introduction_to_the_Wikidata_Query_Service

* **Example queries** <br>
https://www.wikidata.org/wiki/Wikidata:SPARQL_query_service/queries/examples

* **Problematic queries** <br>
https://www.mediawiki.org/wiki/Wikidata_Query_Service/Problematic_queries

* **Wikidata Namespaces** <br>
https://www.wikidata.org/wiki/Help:Namespaces

* **Showcase Queries by Lucas Werkmeister (WMDE) at Wikidata Zurich Training 2019 organized at the University of Zurich** <br>
https://www.wikidata.org/wiki/Wikidata:Events/Wikidata_Zurich_Training2019/Showcase_queries

Querying from Python

* **Wikidata's API** <br>
https://www.wikidata.org/w/api.php

* **Gastrodon Library** <br>
https://github.com/paulhoule/gastrodon

There useful tools to work with SPARQL and RDF data in Python:

* **RDFLib** <br>
https://github.com/RDFLib/rdflib

* **SPARQL Kernel for Jupyter** <br>
https://pypi.org/project/sparqlkernel/

When you try out queries, you often need to look up the names/IDs of items, properties, or maybe you need to look up the way qualifiers and references are stated. In that case, I highly recommend you to look at examples of item descriptions

E.g. https://www.wikidata.org/wiki/Q72
E.g. https://www.wikidata.org/wiki/Q11943 

For a complete description of SPARQL constructs, please check the specification: <br>
https://www.w3.org/TR/sparql11-query/

**Tip!** If you would like to keep up-to-date and discover new very sophisticated SPARQL queries that you can execute, you should not miss the updatey by Wikidata-Facts in Twitter:
https://twitter.com/WikidataFacts



## SPARQL and Wikidata Query Service 

The query service is a Web service that accesses Wikidata's data and can be used by humans through its GUI and by sofware programs to execute data queries written in SPARQL. 
 
 ###  https://query.wikidata.org 

### Exploring SPARQL features


**Get items (ID and label) of a particular type** 
We can use **LIMIT** to restrict the number of results to be obtained (useful for testing/data exploration). 

```
prefix wdt: <http://www.wikidata.org/prop/direct/> 
prefix wd: <http://www.wikidata.org/entity/> 
SELECT ?item ?label
WHERE{
  ?item wdt:P31 wd:Q55488 .
  ?item wdt:P131 wd:Q72 .
  ?item rdfs:label ?label .
  FILTER(LANG(?label) = "en").
}
#LIMIT 2
```

<a href="https://w.wiki/48sK"> Try it here </a>

We can also use Wikidata's integrated service to retrieve labels.

```
prefix wdt: <http://www.wikidata.org/prop/direct/> 
prefix wd: <http://www.wikidata.org/entity/> 
SELECT ?item ?itemLabel
WHERE{
  ?item wdt:P31 wd:Q55488 .
  ?item wdt:P131 wd:Q72 .
  SERVICE wikibase:label {
		bd:serviceParam wikibase:language "en" .
	}
}
#LIMIT 2
```


<a href="https://w.wiki/48tz"> Try it here </a>

**ORDER BY** Sort the results.

```
SELECT ?item ?label
WHERE{
  ?item wdt:P31 wd:Q55488 .
  ?item wdt:P131 wd:Q72 .
  ?item rdfs:label ?label .
  FILTER(LANG(?label) ="en").
     
}
ORDER BY DESC (?label)
```

<a href="https://w.wiki/3Mhz"> Try it here </a>

There might be duplicates in the results. We can add **DISTINCT** 

```
SELECT distinct ?item ?label

```

**Multiple FILTERs and regular expressions** We can add multiple filters and add regular expressions. For example, here, we would like to get only the railway stations whose name start with "W" ("Zurich W"). 

```
SELECT ?item ?label
WHERE{
  ?item wdt:P31 wd:Q55488 .
  ?item wdt:P131 wd:Q72 .
  ?item rdfs:label ?label .
  FILTER(LANG(?label) ="en").
  FILTER(REGEX(?label,"^Zurich W"))   
}
```


<a href="https://w.wiki/7Ngt"> Try it here </a>

**PROPERTY PATHS** If someone described a station using a more specific type of item than train (e.g. dead-end station), that station will not be included in the results. We can use property paths and ask for the items that have been defined as train stations or subclasses of train stations:

```
SELECT ?item ?itemLabel
WHERE{
  ?item wdt:P31/wdt:P279* wd:Q55488 .
   ?item wdt:P131 wd:Q72 .
  SERVICE wikibase:label {
		bd:serviceParam wikibase:language "en" .
	}
}
```

<a href="https://w.wiki/48tf"> Try it here </a>

**BGPs: TRIPLES TREATED AS 'AND'** Sometimes we ask for some information (e.g., opening date) when not all items of that type have such information. So this query will be restrictive and will give us only 16/38 stations in the results (those that have that information): 

```
SELECT ?item ?itemLabel ?openingDate
WHERE{
  ?item wdt:P31/wdt:P279* wd:Q55488 .
   ?item wdt:P131 wd:Q72 .
  ?item wdt:P1619 ?openingDate .
 
  SERVICE wikibase:label {
		bd:serviceParam wikibase:language "en" .
	}
}

```

<a href="https://w.wiki/48th">  Try it here </a>

**OPTIONAL** We can "relax" the condition and ask for the stations, and get the opening date (in case they have it). We will still get the other information in the cases where the station does not have the opening date. 

```
SELECT ?item ?itemLabel ?openingDate
WHERE{
  ?item wdt:P31/wdt:P279* wd:Q55488 .
   ?item wdt:P131 wd:Q72 .
  OPTIONAL{?item wdt:P1619 ?openingDate .}
 
  SERVICE wikibase:label {
		bd:serviceParam wikibase:language "en" .
	}
}
```

<a href="https://w.wiki/48ti">  Try it here </a>

**NOT EXISTS** We can also look for items that are missing a piece of information in their description.

```
SELECT ?item ?itemLabel ?openingDate 
WHERE{
  ?item wdt:P31/wdt:P279* wd:Q55488 .
   ?item wdt:P131 wd:Q72 .
  FILTER NOT EXISTS{?item wdt:P1619 ?openingDate .}

  SERVICE wikibase:label {
        bd:serviceParam wikibase:language "en" .
    }
}

```


<a href="https://w.wiki/48tj">  Try it here </a>

**UNION** If we would like to retrieve the museums and church buildings, we can use UNION.

```
select ?item ?itemLabel ?openingDate
where{
  {?item wdt:P31/wdt:P279* wd:Q33506 .} UNION {?item wdt:P31/wdt:P279* wd:Q16970.} 

   ?item wdt:P131 wd:Q72 .
  OPTIONAL{?item wdt:P1619 ?openingDate .}

  SERVICE wikibase:label {
        bd:serviceParam wikibase:language "en" .
    }
}


```

<a href="https://w.wiki/48r$">  Try it here </a>


**AGGREGATIONS AND GROUP BY** We can compute functions over groups of results (e.g., COUNT to count the number of results, SUM/MIN/MAX/AVG to compute a function over the numeric values in the results) 

```
SELECT (COUNT(?item) AS ?countNumberOfItems)
WHERE{
  ?item wdt:P31 wd:Q55488 .
  ?item wdt:P131 wd:Q72 .
  
}

```

<a href="https://w.wiki/3MiG"> Try it here </a>


```
SELECT ?city (COUNT(?item) AS ?count) 
WHERE{
  ?item wdt:P31 wd:Q55488 .
  ?item wdt:P131 ?city .
  ?city wdt:P31 wd:Q54935504 .

    SERVICE wikibase:label {
		bd:serviceParam wikibase:language "en" .
	}
     }
GROUP BY ?city
```

<a href="https://w.wiki/5jFh"> Try it here </a>

**FEDERATED QUERIES** We can select data from other graphs at the same time. 

```
SELECT DISTINCT ?item ?seaL ?location ?maxEl WHERE { 
  ?item wdt:P31 wd:Q70208 .
  ?item wdt:P1549 ?demonymW .
  ?item wdt:P2044 ?seaL .
 

        
  SERVICE <http://dbpedia.org/sparql> {
     ?location <http://www.w3.org/2002/07/owl#sameAs> ?item .
    ?location <http://dbpedia.org/ontology/demonym> ?demonymD .
    ?location <http://dbpedia.org/ontology/maximumElevation> ?maxEl
   
    
 
  }
  
} LIMIT 100
```

<a href="https://w.wiki/48vB">  Try it here </a>

Other examples: https://www.wikidata.org/wiki/Wikidata:SPARQL_query_service/Federated_queries 

**HAVING** It works similarly to FILTER, but for sets of grouped solutions.

```
SELECT ?city (COUNT(?item) AS ?count) 
WHERE{
  ?item wdt:P31 wd:Q55488 .
  ?item wdt:P131 ?city .
  ?city wdt:P31 wd:Q54935504 .

    SERVICE wikibase:label {
        bd:serviceParam wikibase:language "en" .
    }
     }
GROUP BY ?city
HAVING(COUNT(?item) > 5)
```

<a href="https://w.wiki/5jFk">  Try it here </a>

**MINUS** Removing certain cases from the solution. We would like to remove Bern.

```
SELECT ?city ?cityLabel (COUNT(?item) AS ?count) 
WHERE{
  ?item wdt:P31 wd:Q55488 .
  ?item wdt:P131 ?city .
  ?city wdt:P31 wd:Q54935504 .
  
  MINUS {?item rdfs:Label "Bern"}

    SERVICE wikibase:label {
        bd:serviceParam wikibase:language "en" .
    }
     }
GROUP BY ?city ?cityLabel

```

<a href="https://w.wiki/5jFn">  Try it here </a>

**BIND** Assign values to variables within the query. We can obtain the list of cities that were founded more than ten years ago. 

```

SELECT ?city ?cityLabel ?diffInception
WHERE{
  ?city wdt:P31 wd:Q54935504 .
  ?city wdt:P571 ?inception .

  BIND(2024-YEAR(?inception) AS ?diffInception)
  FILTER (?diffInception>10)
  
  
    SERVICE wikibase:label {
        bd:serviceParam wikibase:language "en" .
    }
     }

#There are many other functions on RDF Terms, numerical values, and strings. 
See also: https://www.w3.org/TR/sparql11-query/
```


<a href="https://w.wiki/BGu5">  Try it here </a>

**SUBQUERIES** Queries embedded into queries. Obtain the Swiss cities whose population is higher than the average population.  

```
SELECT ?city ?cityLabel ?population ?avgP
WHERE{

  ?city wdt:P31 wd:Q54935504 .
  ?city wdt:P1082 ?population.
   
  {
   SELECT (AVG(?populationI) as ?avgP) 
   WHERE{
      ?city wdt:P31 wd:Q54935504 .
      ?city wdt:P1082 ?populationI .
   }
  }
  
  FILTER(?population > ?avgP)

    SERVICE wikibase:label {
        bd:serviceParam wikibase:language "en" .
    }
}

```

<a href="https://w.wiki/5jJN">  Try it here </a>

**OTHER TYPES OF QUERIES** Besides SELECT queries, in SPARQL we can execute DESCRIBE, ASK and CONSTRUCT queries. DESCRIBE queries obtain all the statements about an IRI (e.g., all the statements whose subject is a concrete Wikidata item).

```
describe wd:Q72

```

<a href="https://w.wiki/3Mib"> Try it here. </a>

### Exploring features of the Wikidata Query Service

#### > Get Wikipedia and Wikimedia Commons URLs
**Example**: retrieve the id, label of things located in neighbourhoods of Zurich, with their Wikipedia article and Wikimedia commons URL

```
SELECT ?quartier ?item ?itemLabel ?article ?image ?prop
{?item wdt:P131 ?quartier .
 ?quartier wdt:P31 wd:Q19644586 .
  OPTIONAL {?article schema:about ?item .
    ?article schema:isPartOf <https://en.wikipedia.org/>.
 ?item wdt:P18 ?image}

 SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
```

<a href="https://w.wiki/48to">  Try it here </a>

#### > Queries using / retrieving qualifiers and references --- two valuable features in Wikidata
**Example**: query to the population of the neighbourhoods of Zurich at a particular point in time, and the source of the information.

```
PREFIX pr: <http://www.wikidata.org/prop/reference/>
PREFIX prov: <http://www.w3.org/ns/prov#>

SELECT ?itemLabel ?itemDescription ?item ?population ?time ?ref WHERE {
  ?item wdt:P625 ?coord.
  ?item wdt:P31 wd:Q19644586.
  ?item wdt:P131 wd:Q72.
  OPTIONAL {
    ?item p:P1082 ?statement.
    ?statement prov:wasDerivedFrom ?source.
    ?source pr:P854 ?ref.
    ?statement ps:P1082 ?population.
    ?statement pq:P585 ?time.
  }
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
}
```

<a href="https://w.wiki/48tq">  Try it here </a> 

#### > We can also look for items with statements referencing specific sources

```
SELECT *
WHERE
{
  ?ch wdt:P31 wd:Q54935504 .
  OPTIONAL {?ch wdt:P17 wd:Q39.}
  ?ch ?prop ?statement .
  ?statement prov:wasDerivedFrom ?refnode.
  ?refnode   pr:P854 ?ref.  #pr:P248 #pr:P854
  FILTER (CONTAINS(str(?ref),'statistik.zh.ch')) #statistik.zh.ch

}
```


<a href="https://w.wiki/3Mj3"> Try it here </a>

#### > Display query results in a map
**Example**: query to show the mountains of Switzerland in a map, by height (meters above the sea level).

```
#defaultView:Map
SELECT ?item ?itemLabel ?coord ?height ?layer ?image 
WHERE { ?item wdt:P31 wd:Q8502. ?item wdt:P625 ?coord. 
       ?item wdt:P17 wd:Q39.  ?item wdt:P2044 ?height . 
       BIND( IF(?height < 1000, "<1000 metres", 
                IF(?height < 2000, "1000 - 2000 metres", 
                   IF(?height < 3000, "2000 - 3000 metres", 
                      IF(?height < 4000, "3000 - 4000 metres", "> 4000 metres")))) AS ?layer). 
       OPTIONAL {?item wdt:P18 ?image.} 
                                                            
       SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
````

<a href="https://query.wikidata.org/#%23defaultView%3AMap%0ASELECT%20%3Fitem%20%3FitemLabel%20%3Fcoord%20%3Fheight%20%3Flayer%20%3Fimage%20%0AWHERE%20%7B%20%3Fitem%20wdt%3AP31%20wd%3AQ8502.%20%3Fitem%20wdt%3AP625%20%3Fcoord.%20%0A%20%20%20%20%20%20%20%3Fitem%20wdt%3AP17%20wd%3AQ39.%20%20%3Fitem%20wdt%3AP2044%20%3Fheight%20.%20%0A%20%20%20%20%20%20%20BIND%28%20IF%28%3Fheight%20%3C%201000%2C%20%22%3C1000%20metres%22%2C%20%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20IF%28%3Fheight%20%3C%202000%2C%20%221000%20-%202000%20metres%22%2C%20%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20IF%28%3Fheight%20%3C%203000%2C%20%222000%20-%203000%20metres%22%2C%20%0A%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20IF%28%3Fheight%20%3C%204000%2C%20%223000%20-%204000%20metres%22%2C%20%22%3E%204000%20metres%22%29%29%29%29%20AS%20%3Flayer%29.%20%0A%20%20%20%20%20%20%20OPTIONAL%20%7B%3Fitem%20wdt%3AP18%20%3Fimage.%7D%20%0A%0A%20%20%20%20%20%20%20SERVICE%20wikibase%3Alabel%20%7B%20bd%3AserviceParam%20wikibase%3Alanguage%20%22%5BAUTO_LANGUAGE%5D%2Cen%22.%20%7D%0A%7D">  Try it here </a>

#### > Retrieve _things_ around / in the surrounding area
**Example**: query to retrieve max. 100 libraries around the UZH Zentrum building, with a radius of 2km.

```
#defaultView:Map
SELECT ?thing ?location ?distance ?thingLabel WHERE {
  
    SERVICE wikibase:around { 
      ?thing wdt:P625 ?location . 
      bd:serviceParam wikibase:center "Point(8.548333 47.374722)"^^geo:wktLiteral .
      bd:serviceParam wikibase:radius "2" . 
      bd:serviceParam wikibase:distance ?distance .
    } 
    ?thing wdt:P31/wdt:P279* wd:Q7075
   
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
} ORDER BY ?distance LIMIT 100
````

<a href="https://w.wiki/48ts">  Try it here </a>

**Example**: query to retrieve max. 100 fountains that we know have been added from a specific catalogue, the one maintained by the City of Zurich drinking water supply (https://www.wikidata.org/wiki/Q53629101)

```
#defaultView:Map
SELECT ?thing ?location ?distance ?thingLabel WHERE {
  
    SERVICE wikibase:around { 
      ?thing wdt:P625 ?location . 
      bd:serviceParam wikibase:center "Point(8.548333 47.374722)"^^geo:wktLiteral .
      bd:serviceParam wikibase:radius "2" . 
      bd:serviceParam wikibase:distance ?distance .
    } 
    ?thing p:P528 ?statement . ?statement pq:P972 wd:Q53629101 .
   
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
} ORDER BY ?distance LIMIT 100
```
<a href="https://w.wiki/48tu">  Try it here </a>

#### > Display query results in a bubble-chart

**Example**: query to plot in a bubble chart the number of floods per country in Europe since 1980.

```
#defaultView:BubbleChart
SELECT ?country ?countryLabel (count(distinct ?item) as ?count) 
WHERE
{
  ?item wdt:P31/wdt:P279* wd:Q8068 .#wd:Q8065 .
  ?item wdt:P17 ?country .
  ?country wdt:P30 wd:Q46 .
  ?item wdt:P580 ?starttime .
  FILTER(?starttime >= "1980-00-00T00:00:00Z"^^xsd:dateTime)
  SERVICE wikibase:label { bd:serviceParam wikibase:language "en" }
}
GROUP BY ?country ?countryLabel
```
<a href="https://w.wiki/48tw">  Try it here </a>

## Exercises

Please write a SPARQL query for each of the following query requests: 

**QR1:** We would like to obtain a list of max 10 movies in Wikidata that are of genre either 'drama film' or 'horror film', sorted by publication date. We would like to obtain the ID, the title, and the picture of the movies (if present).

title - property ID P1476 <br/>
image - property ID P18 <br/>
genre - property ID P136 <br/>
drama film - item ID Q130232 <br/>
horror film - item ID Q200092 <br/>
publication date - property ID P577 <br/>


**QR2:** We would like to obtain the number of US movies that Wikidata has per movie genre. 

film - item ID Q11424 <br/>
US- item ID Q30 <br/>
genre - property ID P136 <br/>
country of origin - property ID P495 <br/>


   <font color="#5F6A6A"> <center> CC-BY-SA </center> </font> 
